In [1]:
import pandas as pd
import os

In [2]:
joint_df = pd.read_csv('./joint_clean_data/joint.csv', index_col=0)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Which cols to drop

In [3]:
# remove special info
joint_df.drop(columns=['label_id', 'label_form_id', 'label_user_id', 'label_goal_id', 'label_num', 'goal_form_id',
                      'goal_form_num'], inplace=True)

What gives us the knowledge of who wrote the goal?

Seems like personality will bias our model. We're searching for a more universal solution

In [4]:
# remove personality info
joint_df.drop(columns=['goal_sex', 'goal_age', 'goal_city', 'goal_education', 
                        'goal_activity', 'goal_job'], inplace=True)

#### Split data for convience and than concat

In [5]:
smart_labels = joint_df[['label_specific', 'label_attainable', 'label_time_bound', 'label_unambiguity']]

In [6]:
edu_car_labels = joint_df[['label_education']]
# What to do with career?

In [7]:
abstract_labels = joint_df[['label_abstraction_level']]

In [8]:
topics_labels = joint_df[['label_attractor_knowledge', 'label_attractor_hard_skill', 
                        'label_attractor_soft_skill', 'label_attractor_tool',
                        'label_attractor_community', 'label_attractor_subjectivity',
                        'label_attractor_habits', 'label_attractor_career',
                        'label_attractor_fixing', 'label_attractor_art', 'label_attractor_health']]

In [9]:
nlp_core = joint_df[['goal_id', 'label_questions', 'label_custom_question', 'label_with_context',
                    'goal_name', 'goal_result', 'goal_type', 'goal_first_step',
                    'goal_domain', 'goal_obstacle', 'goal_time']]

#### Prettify labels

In [10]:
# SMART_labels

In [11]:
smart_labels.head()

,label_specific,label_attainable,label_time_bound,label_unambiguity
0,0,1,1,1
1,0,1,1,1
2,1,0,0,0
3,1,1,1,1
4,1,1,1,1


In [12]:
smart_labels.isnull().sum()

label_specific       0
label_attainable     0
label_time_bound     0
label_unambiguity    0
dtype: int64

In [13]:
smart_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18020 entries, 0 to 18019
Data columns (total 4 columns):
label_specific       18020 non-null int64
label_attainable     18020 non-null int64
label_time_bound     18020 non-null int64
label_unambiguity    18020 non-null int64
dtypes: int64(4)
memory usage: 703.9 KB


In [14]:
# Education & Career

In [15]:
edu_car_labels.head()

,label_education
0,0
1,0
2,0
3,1
4,1


In [16]:
edu_car_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18020 entries, 0 to 18019
Data columns (total 1 columns):
label_education    18020 non-null int64
dtypes: int64(1)
memory usage: 281.6 KB


In [17]:
# abstract_labels

In [18]:
abstract_labels.head()

,label_abstraction_level
0,В предметной области
1,В предметной области
2,В предметной области
3,В предметной области
4,В предметной области


In [19]:
abstract_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18020 entries, 0 to 18019
Data columns (total 1 columns):
label_abstraction_level    18020 non-null object
dtypes: object(1)
memory usage: 281.6+ KB


In [20]:
abstract_labels['label_abstraction_level'].value_counts()

Абстрактная             7918
В предметной области    6678
Конкретная              3424
Name: label_abstraction_level, dtype: int64

In [21]:
dummy_abstract = pd.get_dummies(abstract_labels['label_abstraction_level'])

In [22]:
abstract_labels = pd.concat([abstract_labels, dummy_abstract], axis=1)
abstract_labels.drop('label_abstraction_level', axis=1, inplace=True)

In [23]:
abstract_labels.head()

,Абстрактная,В предметной области,Конкретная
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [24]:
abstract_labels.rename(columns={'Абстрактная':'abstract',
                               'В предметной области':'subject'}, inplace=True)

In [25]:
# drop Конкретная as exhaustive value

In [26]:
abstract_labels.drop('Конкретная', axis=1, inplace=True)

In [27]:
abstract_labels.head()

,abstract,subject
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [28]:
# topics_labels

In [29]:
topics_labels.head()

,label_attractor_knowledge,label_attractor_hard_skill,label_attractor_soft_skill,label_attractor_tool,label_attractor_community,label_attractor_subjectivity,label_attractor_habits,label_attractor_career,label_attractor_fixing,label_attractor_art,label_attractor_health
0,Нет,"Не следует из формулировки, но можно предположить","Не следует из формулировки, но можно предположить",Нет,"Не следует из формулировки, но можно предположить",Нет,Да,Да,"Не следует из формулировки, но можно предположить",Нет,NaN
1,"Не следует из формулировки, но можно предположить",Да,Нет,"Не следует из формулировки, но можно предположить",Нет,Да,Нет,Да,Нет,"Не следует из формулировки, но можно предположить",NaN
2,Нет,Да,Да,Нет,Нет,Нет,Нет,Нет,Да,Нет,NaN
3,"Не следует из формулировки, но можно предположить",Да,Нет,Нет,Нет,Да,"Не следует из формулировки, но можно предположить","Не следует из формулировки, но можно предположить","Не следует из формулировки, но можно предположить","Не следует из формулировки, но можно предположить",NaN
4,Нет,Да,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Да,NaN


In [156]:
topics_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18020 entries, 0 to 18019
Data columns (total 11 columns):
label_attractor_knowledge       18020 non-null int64
label_attractor_hard_skill      18020 non-null int64
label_attractor_soft_skill      18020 non-null int64
label_attractor_tool            18020 non-null int64
label_attractor_community       18020 non-null int64
label_attractor_subjectivity    18020 non-null int64
label_attractor_habits          18020 non-null int64
label_attractor_career          18020 non-null int64
label_attractor_fixing          18020 non-null int64
label_attractor_art             18020 non-null int64
label_attractor_health          18007 non-null float64
dtypes: float64(1), int64(10)
memory usage: 1.6 MB


In [30]:
for topic in topics_labels.columns:
    print(topic)
    print()
    print(topics_labels[topic].value_counts())
    print('='*60)

label_attractor_knowledge

Да                                                   8725
Не следует из формулировки, но можно предположить    7401
Нет                                                  1894
Name: label_attractor_knowledge, dtype: int64
label_attractor_hard_skill

Не следует из формулировки, но можно предположить    7943
Да                                                   6731
Нет                                                  3346
Name: label_attractor_hard_skill, dtype: int64
label_attractor_soft_skill

Не следует из формулировки, но можно предположить    8078
Да                                                   5206
Нет                                                  4736
Name: label_attractor_soft_skill, dtype: int64
label_attractor_tool

Нет                                                  16487
Да                                                     767
Не следует из формулировки, но можно предположить      766
Name: label_attractor_tool, dtype: int64
label_attractor

In [31]:
map_ = {'Нет':0, 'Да':1, 'Не следует из формулировки, но можно предположить':1}

In [32]:
for topic in topics_labels.columns:
    topics_labels[topic] = topics_labels[topic].map(map_)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
topics_labels.head()

,label_attractor_knowledge,label_attractor_hard_skill,label_attractor_soft_skill,label_attractor_tool,label_attractor_community,label_attractor_subjectivity,label_attractor_habits,label_attractor_career,label_attractor_fixing,label_attractor_art,label_attractor_health
0,0,1,1,0,1,0,1,1,1,0,NaN
1,1,1,0,1,0,1,0,1,0,1,NaN
2,0,1,1,0,0,0,0,0,1,0,NaN
3,1,1,0,0,0,1,1,1,1,1,NaN
4,0,1,0,0,0,0,0,0,0,1,NaN


#### Prettify NLP-core

In [34]:
nlp_core.head()

,goal_id,label_questions,label_custom_question,label_with_context,goal_name,goal_result,goal_type,goal_first_step,goal_domain,goal_obstacle,goal_time
0,19,NaN,NaN,0,не отстать от `поезда современности`,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать",Творчество и создание нового (Изобразительное ...,Не вижу преград,Не знаю
1,20,Как эта цель связана с окружающими вас людьми?...,NaN,0,Самореализация,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать","Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,Не знаю
2,21,NaN,NaN,0,Улучшенный образ жизни,"Да, четко",Включиться в новый проект/деятельность,Сменить работу,"Иностранные языки, гуманитарные знания и навык...",Не вижу преград,Год
3,22,NaN,NaN,0,обновление устаревшей информации,"Да, четко",Развить имеющиеся знания и навыки,поиск необходимой информации,"Прикладные знания и навыки, ручной труд (инфор...",Не вижу преград,Нет жестких сроков
4,23,NaN,NaN,0,Повышение результативности труда,"Да, четко","Получить знания в новой области, Приобрести но...",С анализа,"Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,Нет жестких сроков


#### Assess what can be dropped

In [35]:
nlp_core.label_questions.isnull().sum()

12013

In [36]:
nlp_core.label_questions.value_counts()

Какой смысл вы в это вкладываете? / Что вы под этим имеете в виду?|А для чего вам это? / Что бы вы хотели делать с этой целью дальше?|Как вы поймете, что вы достигли этой цели? Как будет выглядеть результат?                                                                                                                                                                                                          521
А для чего вам это? / Что бы вы хотели делать с этой целью дальше?|Можете ли вы выделить какие-то этапы для достижения указанной цели?|Как вы поймете, что вы достигли этой цели? Как будет выглядеть результат?                                                                                                                                                                                                         452
Какой смысл вы в это вкладываете? / Что вы под этим имеете в виду?|А для чего вам это? / Что бы вы хотели делать с этой целью дальше?|Можете ли вы выделить какие-то этапы для

In [37]:
nlp_core.label_custom_question.isnull().sum()

16963

In [38]:
nlp_core.label_custom_question.value_counts()

шум                                                                                                 88
Шум                                                                                                 55
Уточните, пожалуйста, Вашу цель:                                                                    54
ШУМ                                                                                                 17
Шум                                                                                                 10
                                                                                                    ..
На какие курсы Вы хотели бы записаться?                                                              1
Какие еще методы Вы хотели бы освоить в ближайшее время? За какие сроки Вы хотели бы и освоить ?     1
В какой сфере новое знание вы хотели бы приобрести?                                                  1
В какой области Вы хотели бы получить знания в первую очередь?           

In [39]:
nlp_core[nlp_core.label_custom_question == 'шум'].head()

,goal_id,label_questions,label_custom_question,label_with_context,goal_name,goal_result,goal_type,goal_first_step,goal_domain,goal_obstacle,goal_time
3612,3602,Другое (что именно?),шум,1,"Предугадать технологии, которые будут актуальн...","Да, четко",Получить знания в новой области,обучение принципам цифровых технологий,"Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,полгода
3626,3621,Другое (что именно?),шум,1,Изучение актуальных новостей,"Да, четко","Включиться в сообщество, наработать новые связ...",изучать новостные порталы каждый день,политика политика,Не вижу преград,Нет жестких сроков
4323,4321,Другое (что именно?),шум,1,интересная работа,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать",экономика и финансы Математика и IT (Работа с ...,Не вижу преград,3 года
4328,4327,Другое (что именно?),шум,1,Победа над проблемами,"Да, четко",Развить имеющиеся знания и навыки,В Энергетических ВУЗ-ах делиться знаниями со с...,Энергетическое превосходство Энергетическое пр...,"Затрудняюсь ответить , но у нас очень длительн...",Необходимо внести в систему образования дополн...
4353,4439,Другое (что именно?),шум,1,плавать,"Да, четко",Получить знания в новой области,"Не знаю, с чего начать",Математика и IT (Программирование и освоение к...,Не вижу преград,Нет жестких сроков


In [40]:
nlp_core.label_with_context.isnull().sum()

0

In [41]:
nlp_core.label_with_context.value_counts()

1    12009
0     6011
Name: label_with_context, dtype: int64

In [42]:
nlp_core.drop('label_with_context', axis=1, inplace=True)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Seems like 'questions' are already reflected in labels

In [43]:
nlp_core.drop(columns=['label_questions', 'label_custom_question'], inplace=True)

In [44]:
# What can be transformed

In [45]:
nlp_core.head()

,goal_id,goal_name,goal_result,goal_type,goal_first_step,goal_domain,goal_obstacle,goal_time
0,19,не отстать от `поезда современности`,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать",Творчество и создание нового (Изобразительное ...,Не вижу преград,Не знаю
1,20,Самореализация,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать","Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,Не знаю
2,21,Улучшенный образ жизни,"Да, четко",Включиться в новый проект/деятельность,Сменить работу,"Иностранные языки, гуманитарные знания и навык...",Не вижу преград,Год
3,22,обновление устаревшей информации,"Да, четко",Развить имеющиеся знания и навыки,поиск необходимой информации,"Прикладные знания и навыки, ручной труд (инфор...",Не вижу преград,Нет жестких сроков
4,23,Повышение результативности труда,"Да, четко","Получить знания в новой области, Приобрести но...",С анализа,"Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,Нет жестких сроков


In [46]:
###########

In [47]:
nlp_core.goal_obstacle.value_counts()

Не вижу преград                                      5763
Лень                                                  148
нехватка времени                                      104
лень                                                  103
Нехватка времени                                       87
                                                     ... 
финансовое вложение                                     1
Финансовое благополучие                                 1
несовпадение сферы деятельности и сферы интересов       1
необходимые вложения                                    1
Сложность применения теории на практике                 1
Name: goal_obstacle, Length: 3152, dtype: int64

Do we need goal obstackles?

In [48]:
nlp_core.goal_result.value_counts()

Да, четко                                                                                                                                                13665
Да, нечетко                                                                                                                                               3925
Нет                                                                                                                                                        428
Принимать все изменения в жизни и работе, связанные с человеческим прогрессом, социально-экономическим положением и популярными тенденциями общества.        1
Постоянно работать над собой. Совершенствовать, впитывать новые знания, поддерживать высоко установленную планку, усваивая современные технологии.           1
Name: goal_result, dtype: int64

In [49]:
correct_goal_result = ['Да, четко', 'Да, нечетко', 'Нет']

In [50]:
df_shifted_cols = nlp_core[nlp_core.goal_result.isin(correct_goal_result)==False]

In [51]:
df_shifted_cols.shape

(2, 8)

In [52]:
def is_trash(x):
    if x in correct_goal_result:
        return 0
    else:
        return 1

In [53]:
nlp_core['remove'] = nlp_core['goal_result'].apply(lambda x: is_trash(x))

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
##########

In [55]:
nlp_core.goal_first_step.value_counts()

Не знаю, с чего начать                                           6681
Обучение                                                           56
обучение                                                           46
Учиться                                                            39
Курсы                                                              29
                                                                 ... 
Обучение на дистанционных курсах                                    1
Нужно создать план, описывающий все подпункты к основной цели       1
Записаться на курсы с преподавателем                                1
Развить умение писать эссе на английском                            1
Coursera и прочие платформы он-лайн образования.                    1
Name: goal_first_step, Length: 10032, dtype: int64

In fact we don't need to know certain steps - but whether there is a plan or not

In [56]:
def you_know(x):
    if 'не знаю' in x.lower():
        return 0
    else:
        return 1

In [57]:
nlp_core['are_first_steps_known'] = nlp_core.loc[:, 'goal_first_step'].apply(lambda x: you_know(x))

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
# nlp_core.drop('goal_first_step', axis=1, inplace=True)

In [59]:
nlp_core.head()

,goal_id,goal_name,goal_result,goal_type,goal_first_step,goal_domain,goal_obstacle,goal_time,remove,are_first_steps_known
0,19,не отстать от `поезда современности`,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать",Творчество и создание нового (Изобразительное ...,Не вижу преград,Не знаю,0,0
1,20,Самореализация,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать","Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,Не знаю,0,0
2,21,Улучшенный образ жизни,"Да, четко",Включиться в новый проект/деятельность,Сменить работу,"Иностранные языки, гуманитарные знания и навык...",Не вижу преград,Год,0,1
3,22,обновление устаревшей информации,"Да, четко",Развить имеющиеся знания и навыки,поиск необходимой информации,"Прикладные знания и навыки, ручной труд (инфор...",Не вижу преград,Нет жестких сроков,0,1
4,23,Повышение результативности труда,"Да, четко","Получить знания в новой области, Приобрести но...",С анализа,"Универсальные компетенции, гибкие навыки (soft...",Не вижу преград,Нет жестких сроков,0,1


In [60]:
######################

In [61]:
nlp_core.goal_time.value_counts()

Нет жестких сроков                                                                                                                                                                                                                                                                  6064
Не знаю, с чего начать                                                                                                                                                                                                                                                              2278
Не знаю                                                                                                                                                                                                                                                                              906
1 год                                                                                                                                                        

In [62]:
# nlp_core.goal_type.value_counts()

In [63]:
# nlp_core.goal_domain.value_counts()

In [64]:
# len(nlp_core.goal_domain.unique())

In [65]:
time_related = ['лет', 'год ', 'меся', 'недел', 'дне', 'года']

In [66]:
def is_time_certain(x):
    x = str(x).lower()
    for i in time_related:
        if i in x:
            return 1
        else:
            continue
    return 0

In [67]:
nlp_core['is_time_certain'] = nlp_core['goal_time'].apply(lambda x: is_time_certain(x))

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
nlp_core[nlp_core['is_time_certain'] == 1].head(20)

,goal_id,goal_name,goal_result,goal_type,goal_first_step,goal_domain,goal_obstacle,goal_time,remove,are_first_steps_known,is_time_certain
9,46,Стать проф фотографом,"Да, четко","Приобрести новые навыки, Включиться в новый пр...",1. Окончить курсы 2.купить технику 3.рассказат...,Творчество и создание нового (Фотография)Униве...,Не вижу преград,3 месяца на окончание курсов и покупку техники,0,1,1
11,65,высшее образование,"Да, четко",Получить официальное образование (с дипломом),поступление в университет,"Личные финансы, работа, бизнес и нормативное р...",Не вижу преград,4 года,0,1,1
13,28,самоопределение,"Да, четко","Получить знания в новой области, Приобрести но...",обучение,"Личные финансы, работа, бизнес и нормативное р...",Не вижу преград,Нет жестких сроков2 года,0,1,1
15,31,Выплатить ипотеку,"Да, четко",Изменить привычки,Сократить личные расходы,"Личные финансы, работа, бизнес и нормативное р...",Уменьшение заработка. Непредвиденные расходы,остался месяц,0,1,1
16,32,Достижение желаемых целей,"Да, нечетко","Получить знания в новой области, Получить офиц...",Обучение,"Личные финансы, работа, бизнес и нормативное р...",Не вижу преград,3 года,0,1,1
38,62,Квалификация,"Да, четко","Получить знания в новой области, Приобрести но...",ВУЗ,"Личные финансы, работа, бизнес и нормативное р...",Не вижу преград,4 года+,0,1,1
40,63,Окончить бакалавриат,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать","Иностранные языки, гуманитарные знания и навык...",Не вижу преград,"1,5 года",0,0,1
41,49,Получить степень магистра,"Да, четко","Получить знания в новой области, Приобрести но...",Поступить в магистратуру,"Иностранные языки, гуманитарные знания и навык...",Не вижу преград,"2,5 года",0,1,1
46,64,Закончить вуз с красным дипломо,"Да, четко",Получить официальное образование (с дипломом),Учиться без долгов,Математика и IT (Экономика),Не вижу преград,"еще 2,5 года",0,1,1
47,69,Получить необходимые знания для повышения квал...,"Да, четко","Получить знания в новой области, Развить имеющ...",записаться на онлайн курс,"Личные финансы, работа, бизнес и нормативное р...",нехватка времени,3 месяца,0,1,1


In [69]:
########################

In [70]:
nlp_core.goal_type.value_counts()

Получить знания в новой области, Приобрести новые навыки, Развить имеющиеся знания и навыки                                                                                                                                                                                        1234
Развить имеющиеся знания и навыки                                                                                                                                                                                                                                                   996
Получить знания в новой области, Приобрести новые навыки, Развить имеющиеся знания и навыки, Включиться в сообщество, наработать новые связи, Включиться в новый проект/деятельность                                                                                                877
Получить знания в новой области                                                                                                                                 

In [71]:
nlp_core.goal_result.value_counts()

Да, четко                                                                                                                                                13665
Да, нечетко                                                                                                                                               3925
Нет                                                                                                                                                        428
Принимать все изменения в жизни и работе, связанные с человеческим прогрессом, социально-экономическим положением и популярными тенденциями общества.        1
Постоянно работать над собой. Совершенствовать, впитывать новые знания, поддерживать высоко установленную планку, усваивая современные технологии.           1
Name: goal_result, dtype: int64

In [72]:
nlp_core.goal_obstacle.value_counts()

Не вижу преград                                      5763
Лень                                                  148
нехватка времени                                      104
лень                                                  103
Нехватка времени                                       87
                                                     ... 
финансовое вложение                                     1
Финансовое благополучие                                 1
несовпадение сферы деятельности и сферы интересов       1
необходимые вложения                                    1
Сложность применения теории на практике                 1
Name: goal_obstacle, Length: 3152, dtype: int64

result, first_steps, time, obstackles - engineering/dummy, than remove

goal_domain - LDA

name + type - noise removal, NLP

### Feature engineering (part 2)

In [73]:
nlp_engineered = nlp_core.copy()

In [74]:
# nlp_engineered.head()

In [75]:
def certainly_imagined(x):
    if ' четко' in x.lower():
        return 1
    else:
        return 0

In [76]:
nlp_engineered['is_certainly_imagined'] = nlp_engineered['goal_result'].apply(lambda x: certainly_imagined(x))

In [77]:
def are_obstackles_expected(x):
    if 'не вижу преград' in str(x).lower():
        return 0
    else:
        return 1

In [78]:
nlp_engineered['are_obstackles_expected'] = nlp_engineered['goal_obstacle'].apply(lambda x: are_obstackles_expected(x))

In [79]:
nlp_engineered[nlp_engineered['are_obstackles_expected'] == 1].head()

,goal_id,goal_name,goal_result,goal_type,goal_first_step,goal_domain,goal_obstacle,goal_time,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected
6,25,Получить новые знания,"Да, четко",Получить знания в новой области,Узнать теороетическую часть,"Иностранные языки, гуманитарные знания и навык...",мало свободного времени,6 мес,0,1,0,1,1
8,27,узнать что -то новое,"Да, четко","Получить знания в новой области, Приобрести но...","Не знаю, с чего начать","Личные финансы, работа, бизнес и нормативное р...",не хватка времени,Нет жестких сроков,0,0,0,1,1
10,47,удержаться на острие технологий,"Да, нечетко","Получить знания в новой области, Приобрести но...",смотрим уроки и читаем книги,"Математика и IT (Работа с данными, Программиро...",занятость,Нет жестких сроков,0,1,0,0,1
15,31,Выплатить ипотеку,"Да, четко",Изменить привычки,Сократить личные расходы,"Личные финансы, работа, бизнес и нормативное р...",Уменьшение заработка. Непредвиденные расходы,остался месяц,0,1,1,1,1
19,35,получить новые знания,"Да, четко",Развить имеющиеся знания и навыки,"Не знаю, с чего начать","Универсальные компетенции, гибкие навыки (soft...",не достаток времени,Нет жестких сроков,0,0,0,1,1


In [80]:
nlp_engineered.drop(columns=['goal_result', 'goal_first_step', 'goal_obstacle', 'goal_time'], inplace=True)

In [81]:
nlp_engineered['space'] = ' '

In [82]:
nlp_engineered['name_type'] = nlp_engineered['goal_name'] + nlp_engineered['space'] + nlp_engineered['goal_type']

In [83]:
nlp_engineered.drop(columns=['goal_type', 'space'], inplace=True)

In [84]:
nlp_engineered.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type
0,19,не отстать от `поезда современности`,Творчество и создание нового (Изобразительное ...,0,0,0,1,0,не отстать от `поезда современности` Получить ...
1,20,Самореализация,"Универсальные компетенции, гибкие навыки (soft...",0,0,0,1,0,Самореализация Получить знания в новой области...
2,21,Улучшенный образ жизни,"Иностранные языки, гуманитарные знания и навык...",0,1,0,1,0,Улучшенный образ жизни Включиться в новый прое...
3,22,обновление устаревшей информации,"Прикладные знания и навыки, ручной труд (инфор...",0,1,0,1,0,обновление устаревшей информации Развить имеющ...
4,23,Повышение результативности труда,"Универсальные компетенции, гибкие навыки (soft...",0,1,0,1,0,Повышение результативности труда Получить знан...


### Noise removal / NLP task

Cleanup pipeline:
1. lower
2. remove special symbols
3. lemmatize
4. remove special parts of speech
5. remove highly frequent

In [85]:
nlp_engineered['goal_domain'] = nlp_engineered['goal_domain'].apply(lambda x: str(x).lower())
nlp_engineered['name_type'] = nlp_engineered['name_type'].apply(lambda x: str(x).lower())
nlp_engineered['goal_name'] = nlp_engineered['goal_name'].apply(lambda x: str(x).lower())

In [86]:
specials_to_remove = [
    '.', '"', "'", '?', '(', ')', '`',
]

In [87]:
def remove_special(x):
    for special in specials_to_remove:
        if special in x:
            x =  x.replace(special, '').strip()
        else:
            pass
    return x

In [88]:
nlp_engineered['goal_domain'] = nlp_engineered['goal_domain'].apply(lambda x: remove_special(x))
nlp_engineered['name_type'] = nlp_engineered['name_type'].apply(lambda x: remove_special(x))
nlp_engineered['goal_name'] = nlp_engineered['goal_name'].apply(lambda x: remove_special(x))

In [89]:
specials_to_replace = [
    '-', '\\', '/', ','
]

In [90]:
def replace_special(x):
    for special in specials_to_replace:
        if special in x:
            x =  x.replace(special, ' ').strip()
        else:
            pass
    return x

In [91]:
nlp_engineered['goal_domain'] = nlp_engineered['goal_domain'].apply(lambda x: replace_special(x))
nlp_engineered['name_type'] = nlp_engineered['name_type'].apply(lambda x: replace_special(x))
nlp_engineered['goal_name'] = nlp_engineered['goal_name'].apply(lambda x: replace_special(x))

In [92]:
nlp_engineered.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...


In [93]:
# !pip install pymorphy2==0.8

In [94]:
# !pip install spacy==2.1.9

In [95]:
# import spacy

In [96]:
# import ru2

In [97]:
def create_list_of_words(x):
    return x.split(' ')

In [98]:
nlp_engineered['goal_domain_LoW'] = nlp_engineered['goal_domain'].apply(lambda x: create_list_of_words(x))
nlp_engineered['name_type_LoW'] = nlp_engineered['name_type'].apply(lambda x: create_list_of_words(x))
nlp_engineered['goal_name_LoW'] = nlp_engineered['goal_name'].apply(lambda x: create_list_of_words(x))

In [99]:
nlp_engineered.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type,goal_domain_LoW,name_type_LoW,goal_name_LoW
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...","[не, отстать, от, поезда, современности, получ...","[не, отстать, от, поезда, современности]"
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...","[самореализация, получить, знания, в, новой, о...",[самореализация]
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...","[улучшенный, образ, жизни, включиться, в, новы...","[улучшенный, образ, жизни]"
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...","[обновление, устаревшей, информации, развить, ...","[обновление, устаревшей, информации]"
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...","[повышение, результативности, труда, получить,...","[повышение, результативности, труда]"


In [100]:
import pymorphy2

In [101]:
morph = pymorphy2.MorphAnalyzer()

In [102]:
senseless_pos = ['NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ']
key_pos = ['NOUN', 'VERB', 'NUMR', 'ADJF', 'ADJS', 'INFN']

In [103]:
def clean_LoW(x):
    clean_LoW = []
    for word in x:
        p = morph.parse(word)[0]
        normal_form = p.normal_form
        pos = p.tag
        stop = 0
        for s_pos in senseless_pos:
            if s_pos in pos:
                stop =1
                break
            else:
                continue
        if stop == 0:
            clean_LoW.append(normal_form)
    return ' '.join(clean_LoW)

In [104]:
nlp_engineered['goal_domain_clean_LoW'] = nlp_engineered['goal_domain_LoW'].apply(lambda x: clean_LoW(x))
nlp_engineered['name_type_clean_LoW'] = nlp_engineered['name_type_LoW'].apply(lambda x: clean_LoW(x))
nlp_engineered['goal_name_clean_LoW'] = nlp_engineered['goal_name_LoW'].apply(lambda x: clean_LoW(x))

In [105]:
nlp_engineered.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type,goal_domain_LoW,name_type_LoW,goal_name_LoW,goal_domain_clean_LoW,name_type_clean_LoW,goal_name_clean_LoW
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...","[не, отстать, от, поезда, современности, получ...","[не, отстать, от, поезда, современности]",творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,отстать поезд современность
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...","[самореализация, получить, знания, в, новой, о...",[самореализация],универсальный компетенция гибкий навык soft s...,самореализация получить знание новый область ...,самореализация
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...","[улучшенный, образ, жизни, включиться, в, новы...","[улучшенный, образ, жизни]",иностранный язык гуманитарный знание навык ин...,улучшить образ жизнь включиться новый проект д...,улучшить образ жизнь
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...","[обновление, устаревшей, информации, развить, ...","[обновление, устаревшей, информации]",прикладной знание навык ручной труд информаци...,обновление устаревший информация развить иметь...,обновление устаревший информация
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...","[повышение, результативности, труда, получить,...","[повышение, результативности, труда]",универсальный компетенция гибкий навык soft s...,повышение результативность труд получить знани...,повышение результативность труд


In [106]:
def clean_LoW_nv(x):
    clean_LoW_nv = []
    for word in x:
        if word.isdigit() == True:
            clean_LoW_nv.append(word)
        else:
            p = morph.parse(word)[0]
            normal_form = p.normal_form
            pos = p.tag
            stop = 0
            for s_pos in key_pos:
                if s_pos in pos:
                    clean_LoW_nv.append(normal_form)
                else:
                    continue
    return ' '.join(clean_LoW_nv)

In [107]:
nlp_engineered['goal_domain_clean_NV_LoW'] = nlp_engineered['goal_domain_LoW'].apply(lambda x: clean_LoW_nv(x))
nlp_engineered['name_type_clean_NV_LoW'] = nlp_engineered['name_type_LoW'].apply(lambda x: clean_LoW_nv(x))
nlp_engineered['goal_name_clean_NV_LoW'] = nlp_engineered['goal_name_LoW'].apply(lambda x: clean_LoW_nv(x))

In [108]:
nlp_engineered.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type,goal_domain_LoW,name_type_LoW,goal_name_LoW,goal_domain_clean_LoW,name_type_clean_LoW,goal_name_clean_LoW,goal_domain_clean_NV_LoW,name_type_clean_NV_LoW,goal_name_clean_NV_LoW
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...","[не, отстать, от, поезда, современности, получ...","[не, отстать, от, поезда, современности]",творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,отстать поезд современность,творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,отстать поезд современность
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...","[самореализация, получить, знания, в, новой, о...",[самореализация],универсальный компетенция гибкий навык soft s...,самореализация получить знание новый область ...,самореализация,универсальный компетенция гибкий навык навык о...,самореализация получить знание новый область п...,самореализация
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...","[улучшенный, образ, жизни, включиться, в, новы...","[улучшенный, образ, жизни]",иностранный язык гуманитарный знание навык ин...,улучшить образ жизнь включиться новый проект д...,улучшить образ жизнь,иностранный язык гуманитарный знание навык ино...,образ жизнь включиться новый проект деятельность,образ жизнь
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...","[обновление, устаревшей, информации, развить, ...","[обновление, устаревшей, информации]",прикладной знание навык ручной труд информаци...,обновление устаревший информация развить иметь...,обновление устаревший информация,прикладной знание навык ручной труд информация...,обновление устаревший информация развить знани...,обновление устаревший информация
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...","[повышение, результативности, труда, получить,...","[повышение, результативности, труда]",универсальный компетенция гибкий навык soft s...,повышение результативность труд получить знани...,повышение результативность труд,универсальный компетенция гибкий навык планиро...,повышение результативность труд получить знани...,повышение результативность труд


In [109]:
from collections import Counter
import operator

In [110]:
def create_bow(array):
    bow = []
    counter = Counter()
    list_ = array.tolist()
    for text in list_:
        list_of_words = text.split(' ')
        for word in list_of_words:
            if word != '' and word != 'nan':
                bow.append(word)
    counter = Counter(bow)
    sorted_bow = sorted(counter.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_bow

In [111]:
topic_bow = create_bow(nlp_engineered['goal_domain_clean_LoW'])

In [112]:
topic_bow

[('навык', 16412),
 ('работа', 8532),
 ('бизнес', 6861),
 ('личный', 6091),
 ('компетенция', 5957),
 ('финансы', 5402),
 ('знание', 4999),
 ('skills', 4869),
 ('soft', 4868),
 ('гибкий', 4857),
 ('нормативный', 4739),
 ('язык', 4660),
 ('новое', 4297),
 ('менеджмент', 4237),
 ('математика', 4096),
 ('гуманитарный', 3956),
 ('it', 3895),
 ('регулирование', 3837),
 ('иностранный', 3620),
 ('универсальный', 3449),
 ('создание', 3421),
 ('освоение', 3348),
 ('развитие', 3265),
 ('мышление', 3141),
 ('планирование', 2937),
 ('самоорганизация', 2930),
 ('тайм', 2930),
 ('ручной', 2907),
 ('осознанность', 2875),
 ('компьютерный', 2699),
 ('программирование', 2698),
 ('командный', 2489),
 ('общение', 2448),
 ('управление', 2414),
 ('самопрезентация', 2365),
 ('зож', 2324),
 ('программа', 2243),
 ('организация', 2217),
 ('процесс', 2199),
 ('рабочий', 2166),
 ('ресурсный', 2155),
 ('эмоция', 2154),
 ('внешность', 2149),
 ('творчество', 2123),
 ('состояние', 2102),
 ('труд', 2029),
 ('дать', 197

In [113]:
goals_bow = create_bow(nlp_engineered['name_type_clean_LoW'])

In [114]:
goals_bow

[('знание', 22209),
 ('навык', 21858),
 ('новое', 18365),
 ('новый', 17535),
 ('получить', 14587),
 ('включиться', 13601),
 ('развить', 10625),
 ('иметься', 10556),
 ('приобрести', 10451),
 ('область', 10201),
 ('деятельность', 7063),
 ('связь', 6955),
 ('сообщество', 6954),
 ('проект', 6933),
 ('наработать', 6929),
 ('образование', 4274),
 ('диплом', 3889),
 ('официальный', 3770),
 ('изменить', 3090),
 ('привычка', 3071),
 ('свой', 1321),
 ('быть', 932),
 ('развитие', 927),
 ('профессиональный', 881),
 ('работа', 826),
 ('повышение', 800),
 ('уровень', 777),
 ('стать', 672),
 ('язык', 651),
 ('получение', 588),
 ('компетенция', 570),
 ('научиться', 550),
 ('рост', 426),
 ('сфера', 409),
 ('английский', 399),
 ('хороший', 389),
 ('повысить', 363),
 ('жизнь', 355),
 ('технология', 355),
 ('который', 348),
 ('найти', 334),
 ('человек', 327),
 ('расширение', 318),
 ('узнать', 311),
 ('обучение', 308),
 ('выучить', 306),
 ('изучение', 299),
 ('умение', 285),
 ('кругозор', 271),
 ('высокий'

In [115]:
topic_nv_bow = create_bow(nlp_engineered['goal_domain_clean_NV_LoW'])

In [116]:
topic_nv_bow

[('навык', 16412),
 ('работа', 8532),
 ('бизнес', 6861),
 ('личный', 6091),
 ('компетенция', 5957),
 ('финансы', 5402),
 ('знание', 4999),
 ('гибкий', 4857),
 ('нормативный', 4739),
 ('язык', 4660),
 ('новое', 4297),
 ('менеджмент', 4237),
 ('математика', 4096),
 ('гуманитарный', 3956),
 ('регулирование', 3837),
 ('иностранный', 3620),
 ('универсальный', 3449),
 ('создание', 3421),
 ('освоение', 3348),
 ('развитие', 3265),
 ('мышление', 3141),
 ('планирование', 2937),
 ('самоорганизация', 2930),
 ('тайм', 2930),
 ('ручной', 2907),
 ('осознанность', 2875),
 ('компьютерный', 2699),
 ('программирование', 2698),
 ('командный', 2489),
 ('общение', 2448),
 ('управление', 2414),
 ('самопрезентация', 2365),
 ('программа', 2243),
 ('организация', 2217),
 ('процесс', 2199),
 ('рабочий', 2166),
 ('ресурсный', 2155),
 ('эмоция', 2154),
 ('внешность', 2149),
 ('творчество', 2123),
 ('состояние', 2102),
 ('труд', 2029),
 ('искусство', 1972),
 ('наука', 1936),
 ('стартап', 1805),
 ('проект', 1761),
 

In [117]:
goals_nv_bow = create_bow(nlp_engineered['name_type_clean_NV_LoW'])

In [118]:
goals_nv_bow

[('знание', 22209),
 ('навык', 21858),
 ('новое', 18365),
 ('новый', 17535),
 ('получить', 14484),
 ('включиться', 13601),
 ('развить', 10625),
 ('приобрести', 10441),
 ('область', 10201),
 ('деятельность', 7063),
 ('связь', 6955),
 ('сообщество', 6954),
 ('проект', 6933),
 ('наработать', 6928),
 ('образование', 4274),
 ('диплом', 3889),
 ('официальный', 3770),
 ('изменить', 3090),
 ('привычка', 3071),
 ('свой', 1321),
 ('быть', 930),
 ('развитие', 927),
 ('профессиональный', 880),
 ('работа', 826),
 ('повышение', 800),
 ('уровень', 777),
 ('стать', 672),
 ('язык', 651),
 ('получение', 588),
 ('компетенция', 570),
 ('научиться', 550),
 ('рост', 426),
 ('сфера', 409),
 ('английский', 399),
 ('повысить', 363),
 ('жизнь', 355),
 ('технология', 355),
 ('который', 348),
 ('найти', 332),
 ('человек', 327),
 ('расширение', 318),
 ('узнать', 311),
 ('обучение', 308),
 ('выучить', 306),
 ('изучение', 299),
 ('умение', 285),
 ('кругозор', 271),
 ('высокий', 262),
 ('время', 262),
 ('хороший', 26

### Feature engineering (part 3)

In [119]:
# nlp_engineered.head()

In [120]:
nlp_feat = nlp_engineered.copy()

In [121]:
# nlp_feat.drop(columns=['goal_domain_LoW', 'name_type_LoW'], inplace=True)

In [122]:
nlp_feat.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type,goal_domain_LoW,name_type_LoW,goal_name_LoW,goal_domain_clean_LoW,name_type_clean_LoW,goal_name_clean_LoW,goal_domain_clean_NV_LoW,name_type_clean_NV_LoW,goal_name_clean_NV_LoW
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...","[не, отстать, от, поезда, современности, получ...","[не, отстать, от, поезда, современности]",творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,отстать поезд современность,творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,отстать поезд современность
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...","[самореализация, получить, знания, в, новой, о...",[самореализация],универсальный компетенция гибкий навык soft s...,самореализация получить знание новый область ...,самореализация,универсальный компетенция гибкий навык навык о...,самореализация получить знание новый область п...,самореализация
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...","[улучшенный, образ, жизни, включиться, в, новы...","[улучшенный, образ, жизни]",иностранный язык гуманитарный знание навык ин...,улучшить образ жизнь включиться новый проект д...,улучшить образ жизнь,иностранный язык гуманитарный знание навык ино...,образ жизнь включиться новый проект деятельность,образ жизнь
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...","[обновление, устаревшей, информации, развить, ...","[обновление, устаревшей, информации]",прикладной знание навык ручной труд информаци...,обновление устаревший информация развить иметь...,обновление устаревший информация,прикладной знание навык ручной труд информация...,обновление устаревший информация развить знани...,обновление устаревший информация
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...","[повышение, результативности, труда, получить,...","[повышение, результативности, труда]",универсальный компетенция гибкий навык soft s...,повышение результативность труд получить знани...,повышение результативность труд,универсальный компетенция гибкий навык планиро...,повышение результативность труд получить знани...,повышение результативность труд


New features:

1. N words original
2. N letters original
3. Average word length
4. N of verbs
5. N of nouns
6. N of numerics
7. N of adjectives
8. N of digits

In [123]:
def word_counter(x):
    return len(x)

In [124]:
nlp_feat['topic_words'] = nlp_feat['goal_domain_LoW'].apply(lambda x: word_counter(x))
nlp_feat['goal_words'] = nlp_feat['name_type_LoW'].apply(lambda x: word_counter(x))

In [125]:
def letters_counter(x):
    counter = int()
    for word in x:
        counter += len(word)
    return counter

In [126]:
nlp_feat['topic_letters'] = nlp_feat['goal_domain_LoW'].apply(lambda x: letters_counter(x))
nlp_feat['goal_letters'] = nlp_feat['name_type_LoW'].apply(lambda x: letters_counter(x))

In [127]:
# nlp_feat.head()

In [128]:
nlp_feat['topic_aver_word_len'] = round(nlp_feat['topic_letters'].div(nlp_feat['topic_words']), 2)
nlp_feat['goal_aver_word_len'] = round(nlp_feat['goal_letters'].div(nlp_feat['goal_words']), 2)

In [129]:
nlp_feat.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type,goal_domain_LoW,...,goal_name_clean_LoW,goal_domain_clean_NV_LoW,name_type_clean_NV_LoW,goal_name_clean_NV_LoW,topic_words,goal_words,topic_letters,goal_letters,topic_aver_word_len,goal_aver_word_len
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...",...,отстать поезд современность,творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,отстать поезд современность,19,32,121,180,6.37,5.62
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...",...,самореализация,универсальный компетенция гибкий навык навык о...,самореализация получить знание новый область п...,самореализация,79,24,435,142,5.51,5.92
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...",...,улучшить образ жизнь,иностранный язык гуманитарный знание навык ино...,образ жизнь включиться новый проект деятельность,образ жизнь,25,8,171,54,6.84,6.75
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...",...,обновление устаревший информация,прикладной знание навык ручной труд информация...,обновление устаревший информация развить знани...,обновление устаревший информация,19,8,106,59,5.58,7.38
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...",...,повышение результативность труд,универсальный компетенция гибкий навык планиро...,повышение результативность труд получить знани...,повышение результативность труд,20,26,134,148,6.70,5.69


In [130]:
def pos_counter(x, pos_to_comp):
    pos_counter = int()
    for word in x:
        p = morph.parse(word)[0]
        pos = p.tag
        for pos_ in pos_to_comp:
            if pos_ in pos:
                pos_counter += 1
            else:
                pass
    return pos_counter

In [131]:
# 'NOUN', 'VERB', 'NUMR', 'ADJF'

In [132]:
nlp_feat['goal_verbs_counter'] = nlp_feat['name_type_LoW'].apply(lambda x: pos_counter(x, ['VERB', 'INFN']))

In [133]:
nlp_feat['goal_nouns_counter'] = nlp_feat['name_type_LoW'].apply(lambda x: pos_counter(x, ['NOUN']))

In [134]:
nlp_feat['goal_numr_counter'] = nlp_feat['name_type_LoW'].apply(lambda x: pos_counter(x, ['NUMR']))

In [135]:
nlp_feat['goal_adj_counter'] = nlp_feat['name_type_LoW'].apply(lambda x: pos_counter(x, ['ADJF', 'ADJS']))

In [136]:
def digit_counter(x):
    digit_counter = int()
    for word in x:
        if word.isdigit() == True:
            digit_counter += 1
    return digit_counter

In [137]:
nlp_feat['goal_digit_counter'] = nlp_feat['name_type_LoW'].apply(lambda x: digit_counter(x))

In [138]:
# p = morph.parse('отстать')[0]
# pos = p.tag
# pos

In [139]:
nlp_feat.head()

,goal_id,goal_name,goal_domain,remove,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,name_type,goal_domain_LoW,...,goal_words,topic_letters,goal_letters,topic_aver_word_len,goal_aver_word_len,goal_verbs_counter,goal_nouns_counter,goal_numr_counter,goal_adj_counter,goal_digit_counter
0,19,не отстать от поезда современности,творчество и создание нового изобразительное и...,0,0,0,1,0,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...",...,32,121,180,6.37,5.62,6,12,0,3,0
1,20,самореализация,универсальные компетенции гибкие навыки soft ...,0,0,0,1,0,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...",...,24,435,142,5.51,5.92,5,10,0,2,0
2,21,улучшенный образ жизни,иностранные языки гуманитарные знания и навык...,0,1,0,1,0,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...",...,8,171,54,6.84,6.75,1,4,0,1,0
3,22,обновление устаревшей информации,прикладные знания и навыки ручной труд информ...,0,1,0,1,0,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...",...,8,106,59,5.58,7.38,1,4,0,1,0
4,23,повышение результативности труда,универсальные компетенции гибкие навыки soft ...,0,1,0,1,0,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...",...,26,134,148,6.70,5.69,5,12,0,1,0


In [140]:
nlp_feat.columns

Index(['goal_id', 'goal_name', 'goal_domain', 'remove',
       'are_first_steps_known', 'is_time_certain', 'is_certainly_imagined',
       'are_obstackles_expected', 'name_type', 'goal_domain_LoW',
       'name_type_LoW', 'goal_name_LoW', 'goal_domain_clean_LoW',
       'name_type_clean_LoW', 'goal_name_clean_LoW',
       'goal_domain_clean_NV_LoW', 'name_type_clean_NV_LoW',
       'goal_name_clean_NV_LoW', 'topic_words', 'goal_words', 'topic_letters',
       'goal_letters', 'topic_aver_word_len', 'goal_aver_word_len',
       'goal_verbs_counter', 'goal_nouns_counter', 'goal_numr_counter',
       'goal_adj_counter', 'goal_digit_counter'],
      dtype='object')

In [141]:
nlp_feat_clean = nlp_feat[['goal_id', 'goal_domain', 'name_type',
                          'goal_domain_LoW', 'name_type_LoW',
                           'goal_domain_clean_LoW', 'name_type_clean_LoW',
                           'goal_domain_clean_NV_LoW', 'name_type_clean_NV_LoW',
                           'remove', 'are_first_steps_known',
                           'is_time_certain', 'is_certainly_imagined', 'are_obstackles_expected',
                           'topic_words',
                           'goal_words', 'topic_letters', 'goal_letters', 'topic_aver_word_len',
                           'goal_aver_word_len', 'goal_verbs_counter', 'goal_nouns_counter',
                           'goal_numr_counter', 'goal_adj_counter', 'goal_digit_counter', 'goal_name',
                           'goal_name_LoW', 'goal_name_clean_LoW', 'goal_name_clean_NV_LoW'
                          ]]

In [142]:
# nlp_feat_clean.head()

In [143]:
nlp_feat_clean.columns = ['goal_id', 'goal_domain_original', 'goal_name_type_original',
                         'goal_domain_LoW', 'goal_name_type_LoW',
                          'goal_domain_no_noise', 'goal_name_type_no_noise',
                          'goal_domain_mean_pos', 'goal_name_type_mean_pos',
                          'remove', 'are_first_steps_known',
                           'is_time_certain', 'is_certainly_imagined', 'are_obstackles_expected',
                           'topic_words',
                           'goal_words', 'topic_letters', 'goal_letters', 'topic_aver_word_len',
                           'goal_aver_word_len', 'goal_verbs_counter', 'goal_nouns_counter',
                           'goal_numr_counter', 'goal_adj_counter', 'goal_digit_counter', 'goal_name',
                          'goal_name_LoW', 'goal_name_clean_LoW', 'goal_name_clean_NV_LoW'
                         ]

In [144]:
nlp_feat_clean.head()

,goal_id,goal_domain_original,goal_name_type_original,goal_domain_LoW,goal_name_type_LoW,goal_domain_no_noise,goal_name_type_no_noise,goal_domain_mean_pos,goal_name_type_mean_pos,remove,...,goal_aver_word_len,goal_verbs_counter,goal_nouns_counter,goal_numr_counter,goal_adj_counter,goal_digit_counter,goal_name,goal_name_LoW,goal_name_clean_LoW,goal_name_clean_NV_LoW
0,19,творчество и создание нового изобразительное и...,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...","[не, отстать, от, поезда, современности, получ...",творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,0,...,5.62,6,12,0,3,0,не отстать от поезда современности,"[не, отстать, от, поезда, современности]",отстать поезд современность,отстать поезд современность
1,20,универсальные компетенции гибкие навыки soft ...,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...","[самореализация, получить, знания, в, новой, о...",универсальный компетенция гибкий навык soft s...,самореализация получить знание новый область ...,универсальный компетенция гибкий навык навык о...,самореализация получить знание новый область п...,0,...,5.92,5,10,0,2,0,самореализация,[самореализация],самореализация,самореализация
2,21,иностранные языки гуманитарные знания и навык...,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...","[улучшенный, образ, жизни, включиться, в, новы...",иностранный язык гуманитарный знание навык ин...,улучшить образ жизнь включиться новый проект д...,иностранный язык гуманитарный знание навык ино...,образ жизнь включиться новый проект деятельность,0,...,6.75,1,4,0,1,0,улучшенный образ жизни,"[улучшенный, образ, жизни]",улучшить образ жизнь,образ жизнь
3,22,прикладные знания и навыки ручной труд информ...,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...","[обновление, устаревшей, информации, развить, ...",прикладной знание навык ручной труд информаци...,обновление устаревший информация развить иметь...,прикладной знание навык ручной труд информация...,обновление устаревший информация развить знани...,0,...,7.38,1,4,0,1,0,обновление устаревшей информации,"[обновление, устаревшей, информации]",обновление устаревший информация,обновление устаревший информация
4,23,универсальные компетенции гибкие навыки soft ...,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...","[повышение, результативности, труда, получить,...",универсальный компетенция гибкий навык soft s...,повышение результативность труд получить знани...,универсальный компетенция гибкий навык планиро...,повышение результативность труд получить знани...,0,...,5.69,5,12,0,1,0,повышение результативности труда,"[повышение, результативности, труда]",повышение результативность труд,повышение результативность труд


### Creating datasets

In [145]:
smart_df = pd.concat([nlp_feat_clean, smart_labels], axis=1)

In [146]:
career = topics_labels[['label_attractor_career']]

In [147]:
edu_car_df = pd.concat([nlp_feat_clean, edu_car_labels, topics_labels], axis=1)

In [148]:
edu_car_df.head()

,goal_id,goal_domain_original,goal_name_type_original,goal_domain_LoW,goal_name_type_LoW,goal_domain_no_noise,goal_name_type_no_noise,goal_domain_mean_pos,goal_name_type_mean_pos,remove,...,label_attractor_hard_skill,label_attractor_soft_skill,label_attractor_tool,label_attractor_community,label_attractor_subjectivity,label_attractor_habits,label_attractor_career,label_attractor_fixing,label_attractor_art,label_attractor_health
0,19,творчество и создание нового изобразительное и...,не отстать от поезда современности получить зн...,"[творчество, и, создание, нового, изобразитель...","[не, отстать, от, поезда, современности, получ...",творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,0,...,1,1,0,1,0,1,1,1,0,NaN
1,20,универсальные компетенции гибкие навыки soft ...,самореализация получить знания в новой области...,"[универсальные, компетенции, , гибкие, навыки,...","[самореализация, получить, знания, в, новой, о...",универсальный компетенция гибкий навык soft s...,самореализация получить знание новый область ...,универсальный компетенция гибкий навык навык о...,самореализация получить знание новый область п...,0,...,1,0,1,0,1,0,1,0,1,NaN
2,21,иностранные языки гуманитарные знания и навык...,улучшенный образ жизни включиться в новый прое...,"[иностранные, языки, , гуманитарные, знания, и...","[улучшенный, образ, жизни, включиться, в, новы...",иностранный язык гуманитарный знание навык ин...,улучшить образ жизнь включиться новый проект д...,иностранный язык гуманитарный знание навык ино...,образ жизнь включиться новый проект деятельность,0,...,1,1,0,0,0,0,0,1,0,NaN
3,22,прикладные знания и навыки ручной труд информ...,обновление устаревшей информации развить имеющ...,"[прикладные, знания, и, навыки, , ручной, труд...","[обновление, устаревшей, информации, развить, ...",прикладной знание навык ручной труд информаци...,обновление устаревший информация развить иметь...,прикладной знание навык ручной труд информация...,обновление устаревший информация развить знани...,0,...,1,0,0,0,1,1,1,1,1,NaN
4,23,универсальные компетенции гибкие навыки soft ...,повышение результативности труда получить знан...,"[универсальные, компетенции, , гибкие, навыки,...","[повышение, результативности, труда, получить,...",универсальный компетенция гибкий навык soft s...,повышение результативность труд получить знани...,универсальный компетенция гибкий навык планиро...,повышение результативность труд получить знани...,0,...,1,0,0,0,0,0,0,0,1,NaN


In [149]:
abstract_df = pd.concat([nlp_feat_clean, abstract_labels], axis=1)

In [155]:
topics_labels.head()

,label_attractor_knowledge,label_attractor_hard_skill,label_attractor_soft_skill,label_attractor_tool,label_attractor_community,label_attractor_subjectivity,label_attractor_habits,label_attractor_career,label_attractor_fixing,label_attractor_art,label_attractor_health
0,0,1,1,0,1,0,1,1,1,0,NaN
1,1,1,0,1,0,1,0,1,0,1,NaN
2,0,1,1,0,0,0,0,0,1,0,NaN
3,1,1,0,0,0,1,1,1,1,1,NaN
4,0,1,0,0,0,0,0,0,0,1,NaN


In [150]:
topics_df = pd.concat([nlp_feat_clean, topics_labels], axis=1)

In [157]:
topics_df.fillna(0, inplace=True)

In [151]:
smart_df.to_csv('./joint_clean_data/smart.csv')

In [152]:
edu_car_df.to_csv('./joint_clean_data/edu_car.csv')

In [153]:
abstract_df.to_csv('./joint_clean_data/abstract.csv')

In [158]:
topics_df.to_csv('./joint_clean_data/topics.csv')